In [1]:
%pip install wandb
%pip install ultralytics
%pip install matplotlib
%pip install numpy
%pip install opencv-python
%pip install inference

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from os \
    import environ

environ['WANDB_DIR'] = '~/wandb'

environ['WANDB_CONSOLE']       = 'off'
environ['WANDB_DISABLE_CODE']  = 'on'
environ['WANDB_NOTEBOOK_NAME'] = 'TrainingOfModel.ipynb'
environ['WANDB_ENTITY']        = 'Designermadsen'

In [3]:
import wandb

from wandb.integration.ultralytics \
    import add_wandb_callback

wandb: WARNING This integration is tested and supported for ultralytics v8.0.238 and below.
wandb: WARNING             Please report any issues to https://github.com/wandb/wandb/issues with the tag `yolov8`.


In [4]:
from ultralytics    \
    import YOLO

from math           \
    import (
        floor
)

from random         \
    import SystemRandom

from os.path        \
    import (
        join
)

In [5]:
ZERO: int = 0
ONE: int = 1

In [6]:
EPOCHS: int = 50
BATCH: int  = 4
PATIENCE: int = 10

IMAGE_WIDTH_SIZE: int = 960
IMAGE_HEIGHT_SIZE: int = 960

IMAGE_DIMENSIONS: tuple = (
    IMAGE_HEIGHT_SIZE, 
    IMAGE_WIDTH_SIZE
)

SEED = None

IS_TO_RESUME: bool = False

MOSAIC_END_AT: int = -ONE

NUMBER_OF_TIMES_TO_ITERATE_TRAINING: int = 1

DEVICES_TO_USE: list = [
    ZERO
]

In [7]:
MODEL_WEIGHT_LOCATION: str | None = None
MODEL_ITERATION_VERSION: int = 2

FINISH_WHEN_DONE: bool = False

In [8]:
def get_weight_location() -> str:
    global MODEL_WEIGHT_LOCATION
    return MODEL_WEIGHT_LOCATION

def set_weight_location(
    value: str
) -> None:
    global MODEL_WEIGHT_LOCATION
    MODEL_WEIGHT_LOCATION = value

def get_current_model_iteration() -> int:
    global MODEL_ITERATION_VERSION
    return MODEL_ITERATION_VERSION

def set_current_model_iteration(
    value: int
) -> None:
    global MODEL_ITERATION_VERSION
    MODEL_ITERATION_VERSION = value

def increment_of_model() -> None:
    global ONE

    set_current_model_iteration(
        get_current_model_iteration() + ONE
    )

def generate_weight_location() -> str:
    weight_path = join(
        str(
            './runs/detect/train' 
        )        
        + 
        str(
            get_current_model_iteration()
        ),
        'weights/best.pt'
    )

    set_weight_location(
        weight_path
    )

    return get_weight_location()

In [9]:
HOME_LOCATION: str = '/home/ai'
DATASET_LOCATION:   str = join(
    HOME_LOCATION,
    'dataset.yaml'
)

YOLO_MODEL_VERSION: str = 'yolo11m.pt'

MODEL_WEIGHT_LOCATION = generate_weight_location()

In [10]:
generator = None
model = None

In [11]:
PROJECT_NAME: str = 'spectre'

wandb.init(
    project     = PROJECT_NAME, 
    job_type    = 'training',
    save_code   = False
)

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find TrainingOfModel.ipynb.
git root error: Cmd('git') failed due to: exit code(128)
  cmdline: git rev-parse --show-toplevel
  stderr: 'fatal: detected dubious ownership in repository at '/mnt/c/Users/Kentv/Desktop/TrainingUltralytics'
To add an exception for this directory, call:

	git config --global --add safe.directory /mnt/c/Users/Kentv/Desktop/TrainingUltralytics'
wandb: WARNING Path ~/wandb/wandb/ wasn't writable, using system temp directory
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING Path ~/wandb/wandb/ wasn't writable, using system temp directory.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [12]:
model = YOLO(
    YOLO_MODEL_VERSION
)

add_wandb_callback(
    model, 
    enable_model_checkpointing = False
)

if not (
    MODEL_WEIGHT_LOCATION is None
):
    model.load(
        get_weight_location()
    )

Transferred 643/649 items from pretrained weights


In [13]:
def getSeed() -> int:
    global SEED
    
    if isSeedNone():
        return generateSeed()

    return SEED

def  setSeed(
    value: int
) -> None:
    global SEED
    SEED = value


def generateGenerator() -> None:
    global generator

    if generator is None:
        generator = SystemRandom()

def isSeedNone() -> bool:
    global SEED
    return SEED is None

def generateSeed() -> None:
    global          \
        SEED,       \
        generator,  \
        ZERO
    
    generateGenerator()

    setSeed(
        generator.randint(
            ZERO, 
            2147483647
        )
    )

    return SEED

In [14]:
if not IS_TO_RESUME:
    generateSeed()

for idx in range(
    NUMBER_OF_TIMES_TO_ITERATE_TRAINING
):
    training_results =                          \
        model.train(
            data         = DATASET_LOCATION, 
            epochs       = EPOCHS,
            batch        = BATCH, 
            imgsz        = IMAGE_DIMENSIONS,
            device       = DEVICES_TO_USE,
            plots        = True,
            patience     = PATIENCE,
            close_mosaic = MOSAIC_END_AT,
            seed         = getSeed(),
            multi_scale  = False,
            resume       = IS_TO_RESUME,
            save         = True
    )
    
    if not (
        int( 
            NUMBER_OF_TIMES_TO_ITERATE_TRAINING 
            - 
            1 
        )
        == 
        idx
    ):
        increment_of_model()

        model.load(
            generate_weight_location()
        )


Ultralytics 8.3.72 🚀 Python-3.12.8 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)
engine/trainer: task=detect, mode=train, model=yolo11m.pt, data=/home/ai/dataset.yaml, epochs=50, time=None, patience=10, batch=4, imgsz=(960, 960), save=True, save_period=-1, cache=False, device=[0], workers=8, project=None, name=train4, exist_ok=False, pretrained=./runs/detect/train2/weights/best.pt, optimizer=auto, verbose=True, seed=1607546386, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=-1, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, sa

train: Scanning /opt/tmod-dataset/training/labels.cache... 2800 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2800/
val: Scanning /opt/tmod-dataset/validation/labels.cache... 1200 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1200/


Plotting labels to runs/detect/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 106 weight(decay=0.0), 113 weight(decay=0.0005), 112 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 960 train, 960 val
Using 8 dataloader workers
Logging results to runs/detect/train4
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      5.28G     0.5779      1.201      1.231         10        960: 100%|██████████| 700/700 [03:41<00:00,  3
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 150/150 [00:2

                   all       1200       1411      0.947       0.89      0.938       0.77
Ultralytics 8.3.72 🚀 Python-3.12.8 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 150/150 [00:2


                   all       1200       1411      0.969      0.944      0.984       0.84
                Cattle         35         35      0.963      0.914       0.98      0.796
                   Dog       1199       1200      0.998      0.991      0.995      0.957
              Humanoid        171        176      0.945      0.926      0.978      0.767


Generating Visualizations for batch-1/1:   0%|          | 0/8 [00:00<?, ?it/s]

Speed: 0.3ms preprocess, 14.7ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/train4


lr/pg0,▃▆███▇▇▇▇▇▇▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
lr/pg1,▃▆███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
lr/pg2,▃▆███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
metrics/mAP50(B),▁▁▃▃▃▄▂▃▃▄▂▄▆▅▆▄▄▅▇▇▇▅▇▆▇▆▆▆▅▇▇▇█▇▇█████
metrics/mAP50-95(B),▂▂▂▁▂▂▃▃▅▄▄▄▄▅▅▃▅▇▅▆▇▅▆▆▆▇▇▆▆▇▇▇█▇▇█████
metrics/precision(B),▄▇▇▇▆▂▄▇▇▆▅▅▁▇▇▄█▅█▅▇▆▇▆▇███▆▃▄▇███▇▅█▇▆
metrics/recall(B),▂▃▁▄▁▄▄▃▂▄▄▄▄▃▃▃▄▄▄▅▅▆▅▅▅▄▅▅▅▅▇▆▆▆▆█▆▇█▆
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▆▆▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▂▂▃▂▂▂▂▁▂▁▁▁▁▁


In [15]:
if FINISH_WHEN_DONE:
    wandb.finish()